# Portfolio Optimization

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim


# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
from scipy.io.wavfile import write

# Batches selection (not important)

In [3]:
def get_batch(dataset, batch_size, seq_length, deltat_in):
    n = dataset.shape[0]
    time_steps = dataset.shape[1]
    idx = np.random.choice(time_steps - seq_length, batch_size)

    '''TODO: construct a list of input sequences for the training batch'''
    input_batch = [dataset[i:i + seq_length] for i in idx]

    '''TODO: construct a list of output sequences for the training batch'''
    output_batch = [dataset[i + deltat_in:i + seq_length + deltat_in] for i in idx]

    # Convert the input and output batches to tensors
    x_batch = torch.tensor(input_batch, dtype=torch.long)
    y_batch = torch.tensor(output_batch, dtype=torch.long)

    return x_batch, y_batch


# Lag-transformation module

In [18]:
class LagTransform(nn.Module):
    def __init__(self, theta, t, input):
        super(LagTransform, self).__init__()
        self.t = t
        self.theta = theta
        self.input = input

    def hyperbolic_weighting(self, theta, t):
        alpha= theta[0] * t**(-theta[1])
        return alpha

    def saturating_exponential_threshold(self, theta, t):
        beta= theta[2] - theta[3]*np.exp(-theta[4]*t)
        return beta


    def forward(self, theta: torch.Tensor, t: torch.Tensor, input : torch.Tensor):
        return self.hyperbolic_weighting(theta,t) / self.saturating_exponential_threshold(theta,t) * np.tanh(self.saturating_exponential_threshold(theta,t) * input)


In [19]:
y=LagTransform.forward(torch.tensor([0.,0.5,1.0,0.5,0.5]), torch.tensor(1.0), torch.tensor([1.0,2.0,3.0]))
print(y)

TypeError: LagTransform.forward() missing 1 required positional argument: 'input'